In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import statistics
import scipy.stats as st
import itertools
from scipy.stats import pearsonr
from scipy.stats import linregress
import seaborn as sns
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import time
import warnings
warnings.simplefilter(action='ignore', category=Warning)


In [ ]:
filepath='/Data/'
EdgeCutoff=300
Mob=pd.read_csv(filepath+'MSA_Dataset_TillJune.csv',index_col=0)

In [ ]:
Mob=Mob[Mob['date']>=20200122]

In [ ]:
#After National Emergency with threshold on confimed case
CtyConfOri=pd.read_csv(filepath+'ArrivalTime/'+'County_ArrT_allrecords_Th1.csv')

CtyConfOri['FIPS']=CtyConfOri['FIPS'].astype(int)
CtyConfOri['OriDays']=CtyConfOri['Days']

In [ ]:
datelistori=list(dict.fromkeys(Mob['date']))
CtyConfOri['Days'].quantile([.25, .5, .75,0.8,0.85,0.9,0.95,0.99])

In [ ]:
from scipy import stats
stats.percentileofscore(CtyConfOri['Days'].dropna(),66, kind='strict')

In [ ]:
Gamma=pd.read_csv('/Gamma_Dataset.csv',index_col=[0])

In [ ]:
start = time.time()
Ctylist=list((set(dict.fromkeys(list(Mob['GEOID1'])))) & (set(dict.fromkeys(list(CtyConfOri['FIPS'])))))
CtyM=len(Ctylist)
StrCtylist=list(map(str,Ctylist))
FourStatelist=[17,6,25,36,53]
FourStateCountylist=set([6013,6037,6067,6075,6081,6085,8031,8037,8097,8097,9001,9009,12011,12086,17031,19103,22051,22071,25009,25017,25021,25025,26099,26125,26163,31055,32003,34003,34013,34017,34023,34025,34029,34031,34039,36059,36061,36071,36087,36103,36119,41067,42091,42101,47037,48201,53033,53053,53061])

f = open('/home/yanchao/COVID19/GlobalOpt_CountyDynamicMobilityCorrelation/Progress/FinalResult/Final_MobRate_AllCtyDist_CountyDynamicMobilityCorrelation_TwoStages_2_DayTh_NewData_NewDate_DiffState_WL0.txt','w')
f.close()
f = open('/home/yanchao/COVID19/GlobalOpt_CountyDynamicMobilityCorrelation/Progress/FinalResult/Final_MobRate_AllCtyDist_CountyDynamicMobilityCorrelation_TwoStages_2_DayTh_NewData_NewDate_DiffState_WL7.txt','w')
f.close()
f = open('/home/yanchao/COVID19/GlobalOpt_CountyDynamicMobilityCorrelation/Progress/FinalResult/Final_MobRate_AllCtyDist_CountyDynamicMobilityCorrelation_TwoStages_2_DayTh_NewData_NewDate_DiffState_WL14.txt','w')
f.close()
collist=list(['sepday','sepdate','SingleSource','MultiSource','R2'])
init=pd.DataFrame(columns=collist)
init.to_csv('/home/yanchao/COVID19/GlobalOpt_CountyDynamicMobilityCorrelation/Progress/FinalResult/Final_MobRate_AllCtyDist_CountyDynamicMobilityCorrelation_TwoStages_2_DayTh_NewData_NewDate_DiffState_WL0.csv',index=False)
init.to_csv('/home/yanchao/COVID19/GlobalOpt_CountyDynamicMobilityCorrelation/Progress/FinalResult/Final_MobRate_AllCtyDist_CountyDynamicMobilityCorrelation_TwoStages_2_DayTh_NewData_NewDate_DiffState_WL7.csv',index=False)
init.to_csv('/home/yanchao/COVID19/GlobalOpt_CountyDynamicMobilityCorrelation/Progress/FinalResult/Final_MobRate_AllCtyDist_CountyDynamicMobilityCorrelation_TwoStages_2_DayTh_NewData_NewDate_DiffState_WL14.csv',index=False)

sourcelength=2
datelistori=list(dict.fromkeys(Mob['date']))
PotentialSingleSourcelist=list((set(dict.fromkeys(list(Mob['GEOID1'])))) & (set(dict.fromkeys(list(CtyConfOri['FIPS'])))) & FourStateCountylist)
PotentialMultiSourcelist=list((set(dict.fromkeys(list(Mob['GEOID1'])))) & (set(dict.fromkeys(list(CtyConfOri['FIPS'])))) & FourStateCountylist)
MultiSourcelistTemp=list(itertools.combinations(PotentialMultiSourcelist, sourcelength))
MultiSourcelist=[item for item in MultiSourcelistTemp if str(item[0])[:-3] != str(item[1])[:-3] ]
EndDay=76
for CtySource in PotentialSingleSourcelist:
    MultiSource=[item for item in MultiSourcelist if CtySource in item]
    CtyDayZero=CtyConfOri[CtyConfOri['FIPS']==CtySource]['Days'].iat[0]
    CtyConfAdj=CtyConfOri.copy()
    CtyConfAdj=CtyConfAdj[CtyConfAdj['Days']<=EndDay] # select date before 3/28
    CtyConfAdj['Days']=CtyConfAdj['Days']-CtyDayZero
    ArrTRec=CtyConfAdj[['FIPS','Days']].set_index('FIPS').to_dict('dict')
    Ctydatelist=datelistori[int(CtyDayZero):]
    for CtySourcelist in MultiSource:
        SecondSource=[x for x in CtySourcelist if x != CtySource][0]
        Cty2DayZero=CtyConfOri[CtyConfOri['FIPS']==SecondSource]['Days'].iat[0]
        if Cty2DayZero<CtyDayZero:
            continue
        startin = time.time()
        Record=pd.DataFrame()
        CtyTargetlist=[i for i in Ctylist if i not in CtySourcelist] 
        sourcelist=CtySourcelist
        targetlist=CtyTargetlist
        # Calculate EF for different source sets
        SingleSrc=pd.DataFrame()
        MultipleSrc1=pd.DataFrame()
        DaysSingle=0
        DaysMultiple=0
        for date in Ctydatelist:
            DayMob=pd.DataFrame()
            DayMob=Mob[Mob['date']==date]
            DayMob['Fraction']=DayMob['counts_x']/DayMob['counts_y'] #Basesum_x
            DayMob['effdist']=1-np.log(DayMob['Fraction'])
            G=nx.from_pandas_edgelist(DayMob,source='GEOID1',target='GEOID2',edge_attr='effdist',create_using=nx.DiGraph())
            
            
            dftemp=pd.DataFrame()
            source=CtySource
            d={'Source':source,'Target':targetlist}
            dftemprec=pd.DataFrame(d)
            length,path = nx.single_source_dijkstra(G,source,cutoff=EdgeCutoff,weight='effdist')
            dftemprec['effdist']=dftemprec['Target'].map(length)
            dftemp=dftemp.append(dftemprec)
            if dftemp.empty:
                DaysSingle=DaysSingle+1
            else:
                dftemp['suminvexp']=1/np.exp(dftemp['effdist'])
                dftemp=dftemp.groupby('Target').sum()
                dftemp['suminvexp'].replace(0, np.nan, inplace=True)
                dftemp=dftemp.reset_index()[['Target','suminvexp']]
                dftemp['CapD']=np.log(CtyM)-np.log(dftemp['suminvexp'])
                dftemp=dftemp.drop(columns=['suminvexp'])
                dftemp['Days']=DaysSingle
                dftemp=dftemp[['Target','CapD','Days']]
                SingleSrc=SingleSrc.append(pd.DataFrame(dftemp))
                DaysSingle=DaysSingle+1
            


            dftemp=pd.DataFrame()
            for source in sourcelist:
                d={'Source':source,'Target':targetlist}
                dftemprec=pd.DataFrame(d)
                length,path = nx.single_source_dijkstra(G,source,cutoff=EdgeCutoff,weight='effdist')
                length_dict = {key: len(value)-1 for key, value in path.items()}
                dftemprec['length']=dftemprec['Target'].map(length_dict)
                dftemprec['effdist']=dftemprec['Target'].map(length)
                dftemp=dftemp.append(dftemprec)

            if dftemp.empty:
                DaysMultiple=DaysMultiple+1
            else:
                minlen=dftemp.groupby(by='Target').min().reset_index()[['Target','length']]#.min().reset_index()
                dftemp['sumexp']=np.exp(dftemp['length']-dftemp['effdist'])
                dftemp=dftemp.groupby('Target').sum()
                dftemp['sumexp'].replace(0, np.nan, inplace=True)
                dftemp=dftemp.reset_index()[['Target','sumexp']]
                dftemp=dftemp.merge(minlen,on='Target')
                dftemp['CapD']=dftemp['length']-np.log(1/CtyM*dftemp['sumexp'])
                dftemp=dftemp.drop(columns=['sumexp'])
                dftemp['Days']=DaysMultiple
                dftemp=dftemp[['Target','CapD','Days']]
                MultipleSrc1=MultipleSrc1.append(pd.DataFrame(dftemp))
                DaysMultiple=DaysMultiple+1
        
        EFRecColList=list(['Target','CapD','Days'])
        EFRecInit=pd.DataFrame(columns=EFRecColList)
        EFRecInit.to_csv('/GlobalOpt_CountyDynamicMobilityCorrelation/Intermediate_Result_EF/SingleSource/SingleSource'+str(CtySource)+'EFRec.csv',index=False)
        EFRecInit.to_csv('/GlobalOpt_CountyDynamicMobilityCorrelation/Intermediate_Result_EF/MultipleSource/MultipleSource'+str(CtySource)+'_'+str(SecondSource)+'EFRec.csv',index=False)

        SingleSrc.to_csv('/GlobalOpt_CountyDynamicMobilityCorrelation/Intermediate_Result_EF/SingleSource/SingleSource'+str(CtySource)+'EFRec.csv',header=False, mode='a',index=False)
        MultipleSrc1.to_csv('/GlobalOpt_CountyDynamicMobilityCorrelation/Intermediate_Result_EF/MultipleSource/MultipleSource'+str(CtySource)+'_'+str(SecondSource)+'EFRec.csv',header=False, mode='a',index=False)
        SingleSrc=pd.read_csv('/GlobalOpt_CountyDynamicMobilityCorrelation/Intermediate_Result_EF/SingleSource/SingleSource'+str(CtySource)+'EFRec.csv')
        MultipleSrc1=pd.read_csv('/GlobalOpt_CountyDynamicMobilityCorrelation/Intermediate_Result_EF/MultipleSource/MultipleSource'+str(CtySource)+'_'+str(SecondSource)+'EFRec.csv')
        Record=pd.DataFrame()        
        for sepday in range(int(CtyDayZero),EndDay+1+1):
            CountyEffdistRecALL=pd.DataFrame()
            
            SingleStage=SingleSrc[SingleSrc['Days']<sepday-CtyDayZero]
            MultipleStage1=MultipleSrc1[MultipleSrc1['Days']>=sepday-CtyDayZero]
            CountyEffdistRecALL=SingleStage.append(MultipleStage1)
            
            CountyEffdistRecALL['ArrT']=CountyEffdistRecALL['Target'].map(ArrTRec['Days'])
            CountyEffdistRecALL=CountyEffdistRecALL.dropna(subset=['ArrT'])
            CountyEffdistRecALL=CountyEffdistRecALL.dropna(subset=['CapD'])
            CtyTDRec=CountyEffdistRecALL[CountyEffdistRecALL['ArrT']<=EndDay-CtyDayZero]
            GammaAdj=Gamma.copy()
            GammaAdj['CumGamma']=GammaAdj[GammaAdj['day']>=int(CtyDayZero)+1]['Gamma'].cumsum()

            GammaAdj['dayadj']=GammaAdj['day']-int(CtyDayZero)+1
            CtyTDRec=CtyTDRec.merge(GammaAdj[['dayadj','CumGamma']],left_on='ArrT',right_on='dayadj',how='left')

            CtyTDRec=CtyTDRec[(CtyTDRec['Days']==CtyTDRec['ArrT'])] # select Ta & (CtyTDRec['Days']>=CtyTDRec['ArrT']-WindowLenCty)
            CtyTDRec=CtyTDRec.reset_index()
            CtyTDRec=CtyTDRec.dropna(subset=['CapD','CumGamma'])
            
            if CtyTDRec.empty:
                continue
            else:

                linregress(CtyTDRec['CapD'],CtyTDRec['CumGamma'])
                regr = linear_model.LinearRegression()

                # Train the model using the training sets
                regr.fit(CtyTDRec[['CapD']], CtyTDRec[['CumGamma']])

                # Make predictions using the testing set
                diabetes_y_pred = regr.predict(CtyTDRec[['CapD']])

                Recordtemp=pd.DataFrame({'sepday':sepday,'sepdate':datelistori[sepday],'SingleSource':CtySource,'MultiSource':[list(CtySourcelist)],'R2':r2_score(CtyTDRec[['CumGamma']], diabetes_y_pred)}) #'Coefficient':regr.coef_[0], 'MSE':mean_squared_error(TDRecDNA[['ArrT']], diabetes_y_pred),
                Record=Record.append(Recordtemp)
                endin = time.time()
                f = open('/GlobalOpt_CountyDynamicMobilityCorrelation/Progress/FinalResult/Final_MobRate_AllCtyDist_CountyDynamicMobilityCorrelation_TwoStages_2_DayTh_NewData_NewDate_DiffState_WL0.txt','a')
                f.write(str(sourcelength)+' '+str(sepday)+' Single:'+str(PotentialSingleSourcelist.index(CtySource)/len(PotentialSingleSourcelist))+' Multi:'+str(MultiSource.index(CtySourcelist)/len(MultiSource))+' Step Time:'+str(endin-startin)+'\n')
                f.close()
                
            startin = time.time()
        Record.to_csv('/GlobalOpt_CountyDynamicMobilityCorrelation/Progress/FinalResult/Final_MobRate_AllCtyDist_CountyDynamicMobilityCorrelation_TwoStages_2_DayTh_NewData_NewDate_DiffState_WL0.csv',header=False, mode='a',index=False)
        Record=pd.DataFrame()
        for sepday in range(int(CtyDayZero),EndDay+1+1):
            CountyEffdistRecALL=pd.DataFrame()
            
            SingleStage=SingleSrc[SingleSrc['Days']<sepday-CtyDayZero]
            MultipleStage1=MultipleSrc1[MultipleSrc1['Days']>=sepday-CtyDayZero]
            CountyEffdistRecALL=SingleStage.append(MultipleStage1)
            
            CountyEffdistRecALL['ArrT']=CountyEffdistRecALL['Target'].map(ArrTRec['Days'])
            CountyEffdistRecALL=CountyEffdistRecALL.dropna(subset=['ArrT'])
            CountyEffdistRecALL=CountyEffdistRecALL.dropna(subset=['CapD'])

            WindowLenCty=7
            CtyTDRec=CountyEffdistRecALL[CountyEffdistRecALL['ArrT']<=EndDay-CtyDayZero]

            CtyTDRec=CtyTDRec[(CtyTDRec['Days']<=CtyTDRec['ArrT']) & (CtyTDRec['Days']>CtyTDRec['ArrT']-WindowLenCty)] # select Ta 
            CtyTDRec=CtyTDRec.groupby(['Target']).mean().reset_index()
            GammaAdj=Gamma.copy()
            GammaAdj['CumGamma']=GammaAdj[GammaAdj['day']>=int(CtyDayZero)+1]['Gamma'].cumsum()
            GammaAdj['dayadj']=GammaAdj['day']-int(CtyDayZero)+1
            CtyTDRec=CtyTDRec.merge(GammaAdj[['dayadj','CumGamma']],left_on='ArrT',right_on='dayadj',how='left')
            CtyTDRec=CtyTDRec.dropna(subset=['CapD','CumGamma'])
            
            if CtyTDRec.empty:
                continue
            else:

                linregress(CtyTDRec['CapD'],CtyTDRec['CumGamma'])
                regr = linear_model.LinearRegression()

                # Train the model using the training sets
                regr.fit(CtyTDRec[['CapD']], CtyTDRec[['CumGamma']])

                # Make predictions using the testing set
                diabetes_y_pred = regr.predict(CtyTDRec[['CapD']])

                Recordtemp=pd.DataFrame({'sepday':sepday,'sepdate':datelistori[sepday],'SingleSource':CtySource,'MultiSource':[list(CtySourcelist)],'R2':r2_score(CtyTDRec[['CumGamma']], diabetes_y_pred)}) #'Coefficient':regr.coef_[0], 'MSE':mean_squared_error(TDRecDNA[['ArrT']], diabetes_y_pred),
                Record=Record.append(Recordtemp)
                endin = time.time()
                f = open('/GlobalOpt_CountyDynamicMobilityCorrelation/Progress/FinalResult/Final_MobRate_AllCtyDist_CountyDynamicMobilityCorrelation_TwoStages_2_DayTh_NewData_NewDate_DiffState_WL7.txt','a')
                f.write(str(sourcelength)+' '+str(sepday)+' Single:'+str(PotentialSingleSourcelist.index(CtySource)/len(PotentialSingleSourcelist))+' Multi:'+str(MultiSource.index(CtySourcelist)/len(MultiSource))+' Step Time:'+str(endin-startin)+'\n')
                f.close()
                
            startin = time.time()
        Record.to_csv('/GlobalOpt_CountyDynamicMobilityCorrelation/Progress/FinalResult/Final_MobRate_AllCtyDist_CountyDynamicMobilityCorrelation_TwoStages_2_DayTh_NewData_NewDate_DiffState_WL7.csv',header=False, mode='a',index=False)
        Record=pd.DataFrame()
        for sepday in range(int(CtyDayZero),EndDay+1+1):
            CountyEffdistRecALL=pd.DataFrame()
            
            SingleStage=SingleSrc[SingleSrc['Days']<sepday-CtyDayZero]
            MultipleStage1=MultipleSrc1[MultipleSrc1['Days']>=sepday-CtyDayZero]
            CountyEffdistRecALL=SingleStage.append(MultipleStage1)
            
            CountyEffdistRecALL['ArrT']=CountyEffdistRecALL['Target'].map(ArrTRec['Days'])
            CountyEffdistRecALL=CountyEffdistRecALL.dropna(subset=['ArrT'])
            CountyEffdistRecALL=CountyEffdistRecALL.dropna(subset=['CapD'])

            WindowLenCty=14
            CtyTDRec=CountyEffdistRecALL[CountyEffdistRecALL['ArrT']<=EndDay-CtyDayZero]

            CtyTDRec=CtyTDRec[(CtyTDRec['Days']<=CtyTDRec['ArrT']) & (CtyTDRec['Days']>CtyTDRec['ArrT']-WindowLenCty)] # select Ta 
            CtyTDRec=CtyTDRec.groupby(['Target']).mean().reset_index()
            GammaAdj=Gamma.copy()
            GammaAdj['CumGamma']=GammaAdj[GammaAdj['day']>=int(CtyDayZero)+1]['Gamma'].cumsum()
            GammaAdj['dayadj']=GammaAdj['day']-int(CtyDayZero)+1
            CtyTDRec=CtyTDRec.merge(GammaAdj[['dayadj','CumGamma']],left_on='ArrT',right_on='dayadj',how='left')

            CtyTDRec=CtyTDRec.dropna(subset=['CapD','CumGamma'])
            
            if CtyTDRec.empty:
                continue
            else:

                linregress(CtyTDRec['CapD'],CtyTDRec['CumGamma'])
                regr = linear_model.LinearRegression()

                # Train the model using the training sets
                regr.fit(CtyTDRec[['CapD']], CtyTDRec[['CumGamma']])

                # Make predictions using the testing set
                diabetes_y_pred = regr.predict(CtyTDRec[['CapD']])

                Recordtemp=pd.DataFrame({'sepday':sepday,'sepdate':datelistori[sepday],'SingleSource':CtySource,'MultiSource':[list(CtySourcelist)],'R2':r2_score(CtyTDRec[['CumGamma']], diabetes_y_pred)}) #'Coefficient':regr.coef_[0], 'MSE':mean_squared_error(TDRecDNA[['ArrT']], diabetes_y_pred),
                Record=Record.append(Recordtemp)
                endin = time.time()
                f = open('/GlobalOpt_CountyDynamicMobilityCorrelation/Progress/FinalResult/Final_MobRate_AllCtyDist_CountyDynamicMobilityCorrelation_TwoStages_2_DayTh_NewData_NewDate_DiffState_WL14.txt','a')
                f.write(str(sourcelength)+' '+str(sepday)+' Single:'+str(PotentialSingleSourcelist.index(CtySource)/len(PotentialSingleSourcelist))+' Multi:'+str(MultiSource.index(CtySourcelist)/len(MultiSource))+' Step Time:'+str(endin-startin)+'\n')
                f.close()
                
            startin = time.time()
        Record.to_csv('/GlobalOpt_CountyDynamicMobilityCorrelation/Progress/FinalResult/Final_MobRate_AllCtyDist_CountyDynamicMobilityCorrelation_TwoStages_2_DayTh_NewData_NewDate_DiffState_WL14.csv',header=False, mode='a',index=False)
                
end = time.time()
print(end - start)

f = open('/GlobalOpt_CountyDynamicMobilityCorrelation/Progress/FinalResult/Final_MobRate_AllCtyDist_CountyDynamicMobilityCorrelation_TwoStages_2_DayTh_NewData_NewDate_DiffState_time.txt','w')
f.write('Total Time:'+str(end-start))
f.close()

In [ ]:
print(end - start)